In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import csv
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield instance

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# ONE RUN
#############################################

sumo_RL = simulation.simulator(
                     connection_label = "lonely_worker",
                     q_network_type = 'simple',
                     target_q_network_type = 'simple',
                     gamma = 0.99,
                     target_update_freq = 30000,
                     train_freq = 4,
                     num_burn_in = 300,
                     batch_size = 40,
                     optimizer = optimizers.RMSprop(lr=0.001, rho=0.95),
                     loss_func = "mse",
                     max_ep_length = 1000,
                     experiment_id = "Long_Simulation",
                     model_checkpoint = True,
                     opt_metric = None,
                     
                    # environment parameters
                     net_file = "cross.net.xml",
                     route_file = "cross.rou.xml",
                     demand = "nominal",
                     state_shape = (1,11),
                     num_actions = 2,
                     use_gui = False,
                     delta_time = 10,
                 
                    # memory parameters
                     max_size = 100000,
                     
                    # additional parameters
                 
                     policy = "linDecEpsGreedy",
                     eps = 0.1,
                     num_episodes = 1000,
                     monitoring = True)
#agent.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")
# agent.evaluate(runs=1, use_gui= False)

In [ ]:
agent.evaluate(runs=1, use_gui= False)

In [ ]:
# GRIDSEARCH
###############################################################

exp_id = "Nominal_Framework"

# Framework_1 --> Expplain RL current arquitecture

param = {
    "batch_size" : [30,70], # 50,60],
    "target_update_frequency" : [1000,50000,10000],# 20000,30000]
    "gamma" : [0.98, 0.99],#,0.99, 0.995
    "train_freq" : [2,5],#,3,4,
    "max_size" : [100000,1000000], #50000
    "max_ep_length" : [500,1000]#,4000]
}

param_grid = iter_params(**param)

def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all objects
    
    # print('Run', position + 1, '-- parameters', args)
    
    agent = simulation.simulator(connection_label = position + 1, #"lonely_worker",
                                 q_network_type = 'simple',
                                 target_q_network_type = 'simple',
                                 gamma = args[2], #0.99
                                 target_update_freq = args[1], #10000,
                                 train_freq = args[3], #3
                                 num_burn_in = 300,
                                 batch_size = args[0], #32
                                 optimizer = optimizers.RMSprop(lr=0.001, rho=0.95),
                                 loss_func = "mse",
                                 max_ep_length = args[4], #1000
                                 experiment_id = exp_id,#"Exp_1",
                                 model_checkpoint = True,
                                 opt_metric = None,
                                 
                                # environment parameters
                                 net_file = "cross.net.xml",
                                 route_file = "cross.rou.xml",
                                 network_dir = "./network",
                                 demand = "nominal", #"nominal",
                                 state_shape = (1,11),
                                 num_actions = 2,
                                 use_gui = False,
                                 delta_time = 10,
                 
                                # memory parameters
                                 max_size = args[4],# 100000,
                                 
                                 # additional parameters
                                 policy = "linDecEpsGreedy",
                                 eps = 0.1,
                                 num_episodes = 700, #2,
                                 monitoring = True)

 
    # print("training agent", position + 1)
    
    agent.train()
    # print("evaluating agent", position + 1)
    result = agent.evaluate(runs = 3)

    return (multiprocessing.current_process().name, position + 1, args, result)


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
    
    
    with open(os.path.join("./logs",exp_id,"GS_results"), "w",newline='') as file:
        writer = csv.writer(file, dialect = 'excel')
        writer.writerow(["Process", "Run", "Parameters", "Result"])
    
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",exp_id,"GS_results"), "a",newline='') as file:
            writer = csv.writer(file, dialect = 'excel')
            writer.writerow(results[-1])                
        print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
Run 2 -- running episode 1 / 700
Run 5 -- running episode 1 / 700
Run 6 -- running episode 1 / 700
Run 1 -- running episode 1 / 700
Run 8 -- running episode 1 / 700
Run 3 -- running episode 1 / 700
Run 7 -- running episode 1 / 700
Run 4 -- running episode 1 / 700
Run 5 -- running episode 2 / 700
Run 6 -- running episode 2 / 700
Run 2 -- running episode 2 / 700
Run 1 -- running episode 2 / 700
Run 5 -- running episode 3 / 700
Run 6 -- running episode 3 / 700
Run 8 -- running episode 2 / 700
Run 7 -- running episode 2 / 700
Run 2 -- running episode 3 / 700
Run 3 -- running episode 2 / 700
Run 4 -- running episode 2 / 700
Run 1 -- running episode 3 / 700
Run 5 -- running episode 4 / 700
Run 6 -- running episode 4 / 700
Run 8 -- running episode 3 / 700
Run 7 -- running episode 3 / 700
Run 2 -- running episode 4 / 700
Run 3 -- runnin

Run 7 -- running episode 32 / 700
Run 4 -- running episode 27 / 700
Run 6 -- running episode 34 / 700
Run 2 -- running episode 29 / 700
Run 1 -- running episode 29 / 700
Run 8 -- running episode 33 / 700
Run 5 -- running episode 35 / 700
Run 3 -- running episode 28 / 700
Run 7 -- running episode 33 / 700
Run 6 -- running episode 35 / 700
Run 4 -- running episode 28 / 700
Run 2 -- running episode 30 / 700
Run 1 -- running episode 30 / 700
Run 8 -- running episode 34 / 700
Run 5 -- running episode 36 / 700
Run 7 -- running episode 34 / 700
Run 6 -- running episode 36 / 700
Run 3 -- running episode 29 / 700
Run 4 -- running episode 29 / 700
Run 2 -- running episode 31 / 700
Run 8 -- running episode 35 / 700
Run 1 -- running episode 31 / 700
Run 5 -- running episode 37 / 700
Run 6 -- running episode 37 / 700
Run 7 -- running episode 35 / 700
Run 3 -- running episode 30 / 700
Run 8 -- running episode 36 / 700
Run 5 -- running episode 38 / 700
Run 2 -- running episode 32 / 700
Run 4 -- runni

Run 5 -- running episode 67 / 700
Run 2 -- running episode 57 / 700
Run 4 -- running episode 54 / 700
Run 6 -- running episode 67 / 700
Run 8 -- running episode 65 / 700
Run 7 -- running episode 65 / 700
Run 1 -- running episode 57 / 700
Run 5 -- running episode 68 / 700
Run 3 -- running episode 56 / 700
Run 6 -- running episode 68 / 700
Run 2 -- running episode 58 / 700
Run 8 -- running episode 66 / 700
Run 7 -- running episode 66 / 700
Run 4 -- running episode 55 / 700
Run 1 -- running episode 58 / 700
Run 5 -- running episode 69 / 700
Run 6 -- running episode 69 / 700
Run 3 -- running episode 57 / 700
Run 8 -- running episode 67 / 700
Run 7 -- running episode 67 / 700
Run 2 -- running episode 59 / 700
Run 4 -- running episode 56 / 700
Run 1 -- running episode 59 / 700
Run 5 -- running episode 70 / 700
Run 6 -- running episode 70 / 700
Run 8 -- running episode 68 / 700
Run 7 -- running episode 68 / 700
Run 3 -- running episode 58 / 700
Run 2 -- running episode 60 / 700
Run 4 -- runni

Run 5 -- running episode 99 / 700
Run 1 -- running episode 85 / 700
Run 6 -- running episode 99 / 700
Run 4 -- running episode 82 / 700
Run 7 -- running episode 97 / 700
Run 8 -- running episode 97 / 700
Run 2 -- running episode 86 / 700
Run 3 -- running episode 84 / 700
Run 5 -- running episode 100 / 700
Run 1 -- running episode 86 / 700
Run 6 -- running episode 100 / 700
Run 7 -- running episode 98 / 700
Run 4 -- running episode 83 / 700
Run 8 -- running episode 98 / 700
Run 2 -- running episode 87 / 700
Run 5 -- running episode 101 / 700
Run 1 -- running episode 87 / 700
Run 3 -- running episode 85 / 700
Run 7 -- running episode 99 / 700
Run 6 -- running episode 101 / 700
Run 4 -- running episode 84 / 700
Run 8 -- running episode 99 / 700
Run 2 -- running episode 88 / 700
Run 1 -- running episode 88 / 700
Run 3 -- running episode 86 / 700
Run 5 -- running episode 102 / 700
Run 7 -- running episode 100 / 700
Run 8 -- running episode 100 / 700
Run 6 -- running episode 102 / 700
Run 4 

Run 3 -- running episode 114 / 700
Run 6 -- running episode 127 / 700
Run 1 -- running episode 117 / 700
Run 2 -- running episode 117 / 700
Run 4 -- running episode 113 / 700
Run 8 -- running episode 125 / 700
Run 5 -- running episode 127 / 700
Run 1 -- running episode 118 / 700
Run 7 -- running episode 126 / 700
Run 3 -- running episode 115 / 700
Run 6 -- running episode 128 / 700
Run 2 -- running episode 118 / 700
Run 1 -- running episode 119 / 700
Run 3 -- running episode 116 / 700
Run 5 -- running episode 128 / 700
Run 8 -- running episode 126 / 700
Run 4 -- running episode 114 / 700
Run 7 -- running episode 127 / 700
Run 6 -- running episode 129 / 700
Run 1 -- running episode 120 / 700
Run 3 -- running episode 117 / 700
Run 2 -- running episode 119 / 700
Run 4 -- running episode 115 / 700
Run 8 -- running episode 127 / 700
Run 5 -- running episode 129 / 700
Run 7 -- running episode 128 / 700
Run 6 -- running episode 130 / 700
Run 4 -- running episode 116 / 700
Run 1 -- running epi

...

Run 37 -- running episode 196 / 700
Run 35 -- running episode 415 / 700
Run 36 -- running episode 390 / 700
Run 37 -- running episode 197 / 700
Run 38 -- running episode 106 / 700
Run 34 -- running episode 282 / 700
Run 35 -- running episode 416 / 700
Run 36 -- running episode 391 / 700
Run 37 -- running episode 198 / 700
Run 38 -- running episode 107 / 700
Run 35 -- running episode 417 / 700
Run 36 -- running episode 392 / 700
Run 37 -- running episode 199 / 700
Run 29 -- running episode 477 / 700
Run 38 -- running episode 108 / 700
Run 35 -- running episode 418 / 700
Run 37 -- running episode 200 / 700
Run 36 -- running episode 393 / 700
Run 23 -- running episode 571 / 700
Run 38 -- running episode 109 / 700
Run 37 -- running episode 201 / 700
Run 35 -- running episode 419 / 700
Run 36 -- running episode 394 / 700
Run 37 -- running episode 202 / 700
Run 36 -- running episode 395 / 700
Run 38 -- running episode 110 / 700
Run 35 -- running episode 420 / 700
Run 33 -- running episode 25

Run 37 -- running episode 255 / 700
Run 38 -- running episode 142 / 700
Run 23 -- running episode 580 / 700
Run 36 -- running episode 445 / 700
Run 37 -- running episode 256 / 700
Run 29 -- running episode 486 / 700
Run 35 -- running episode 459 / 700
Run 36 -- running episode 446 / 700
Run 37 -- running episode 257 / 700
Run 35 -- running episode 460 / 700
Run 34 -- running episode 290 / 700
Run 36 -- running episode 447 / 700
Run 35 -- running episode 461 / 700
Run 38 -- running episode 143 / 700
Run 35 -- running episode 462 / 700
Run 36 -- running episode 448 / 700
Run 35 -- running episode 463 / 700
Run 37 -- running episode 258 / 700
Run 36 -- running episode 449 / 700
Run 33 -- running episode 267 / 700
Run 38 -- running episode 144 / 700
Run 35 -- running episode 464 / 700
Run 37 -- running episode 259 / 700
Run 23 -- running episode 581 / 700
Run 29 -- running episode 487 / 700
Run 36 -- running episode 450 / 700
Run 37 -- running episode 260 / 700
Run 35 -- running episode 46

Run 38 -- running episode 162 / 700
Run 35 -- running episode 544 / 700
Run 37 -- running episode 298 / 700
Run 38 -- running episode 163 / 700
Run 23 -- running episode 594 / 700
Run 35 -- running episode 545 / 700
Run 37 -- running episode 299 / 700
Run 35 -- running episode 546 / 700
Run 37 -- running episode 300 / 700
Run 37 -- running episode 301 / 700
Run 36 -- running episode 472 / 700
Run 35 -- running episode 547 / 700
Run 37 -- running episode 302 / 700
Run 38 -- running episode 164 / 700
Run 35 -- running episode 548 / 700
Run 37 -- running episode 303 / 700
Run 29 -- running episode 501 / 700
Run 38 -- running episode 165 / 700
Run 33 -- running episode 279 / 700
Run 37 -- running episode 304 / 700
Run 35 -- running episode 549 / 700
Run 35 -- running episode 550 / 700
Run 34 -- running episode 302 / 700
Run 23 -- running episode 595 / 700
Run 36 -- running episode 473 / 700
Run 35 -- running episode 551 / 700
Run 38 -- running episode 166 / 700
Run 35 -- running episode 55

Run 23 -- running episode 607 / 700
Run 37 -- running episode 342 / 700
Run 35 -- running episode 611 / 700
Run 37 -- running episode 343 / 700
Run 35 -- running episode 612 / 700
Run 37 -- running episode 344 / 700
Run 37 -- running episode 345 / 700
Run 35 -- running episode 613 / 700
Run 33 -- running episode 290 / 700
Run 37 -- running episode 346 / 700
Run 34 -- running episode 313 / 700
Run 38 -- running episode 213 / 700
Run 35 -- running episode 614 / 700
Run 36 -- running episode 489 / 700
Run 35 -- running episode 615 / 700
Run 29 -- running episode 515 / 700
Run 23 -- running episode 608 / 700
Run 35 -- running episode 616 / 700
Run 37 -- running episode 347 / 700
Run 35 -- running episode 617 / 700
Run 37 -- running episode 348 / 700
Run 38 -- running episode 214 / 700
Run 35 -- running episode 618 / 700
Run 33 -- running episode 291 / 700
Run 37 -- running episode 349 / 700
Run 34 -- running episode 314 / 700
Run 37 -- running episode 350 / 700
Run 35 -- running episode 61

Run 29 -- running episode 530 / 700
Run 34 -- running episode 326 / 700
Run 37 -- running episode 416 / 700
Run 37 -- running episode 417 / 700
Run 35 -- running episode 676 / 700
Run 36 -- running episode 503 / 700
Run 38 -- running episode 229 / 700
Run 37 -- running episode 418 / 700
Run 37 -- running episode 419 / 700
Run 37 -- running episode 420 / 700
Run 23 -- running episode 624 / 700
Run 35 -- running episode 677 / 700
Run 37 -- running episode 421 / 700
Run 37 -- running episode 422 / 700
Run 35 -- running episode 678 / 700
Run 29 -- running episode 531 / 700
Run 33 -- running episode 304 / 700
Run 37 -- running episode 423 / 700
Run 37 -- running episode 424 / 700
Run 35 -- running episode 679 / 700
Run 34 -- running episode 327 / 700
Run 37 -- running episode 425 / 700
Run 38 -- running episode 230 / 700
Run 36 -- running episode 504 / 700
Run 37 -- running episode 426 / 700
Run 35 -- running episode 680 / 700
Run 37 -- running episode 427 / 700
Run 23 -- running episode 62

In [ ]:
winner